In [ ]:

import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

import os
os.sys.path.append('/home/schirrmr/braindecode/code/fbcsp/')
os.sys.path.append('/home/schirrmr/braindecode/code/braindecode/')

%load_ext autoreload
%autoreload 2
%cd /home/schirrmr/

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7

import logging
log = logging.getLogger()
log.setLevel('DEBUG')

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
set_loader = BBCIDataset('data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat',
                        load_sensor_names=['C3', 'C4', 'C1', 'C2', 'CPz', 'F3', 'F4', 'Fz'])
cnt = set_loader.load()
cnt = cnt.drop_channels(['STI 014'])
eog_cnt = BBCIDataset('data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat',
                        load_sensor_names=['EOGh', 'EOGv']).load()
eog_cnt = eog_cnt.drop_channels(['STI 014'])

In [ ]:
from braindecode.mne_ext.signalproc import mne_apply
from braindecode.datautil.signalproc import bandpass_cnt

import numpy as np
def car_cnt(cnt):
    return mne_apply(lambda x: x - np.mean(x,axis=0, keepdims=True), cnt)

cnt = car_cnt(cnt)
cnt = mne_apply(lambda x: bandpass_cnt(x, 0.5,150.0, cnt.info['sfreq']), cnt)

In [ ]:
from fbcsp.experiment import CSPExperiment
from collections import OrderedDict
name_to_start_codes = OrderedDict([('Right', 1), ('Left', 2), ('Rest',3)])
epoch_ival_ms = [0,4000]

In [ ]:
from fbcsp.clean import EOGMaxMinCleaner, VarCleaner, apply_multiple_cleaners


cleaners = [EOGMaxMinCleaner(eog_cnt, epoch_ival_ms, name_to_start_codes, threshold=600),
           VarCleaner(whisker_percent=10, whisker_length=3)]
cleaned_cnt = apply_multiple_cleaners(cnt, epoch_ival_ms, name_to_start_codes, cleaners)

In [ ]:
exp = CSPExperiment(cleaned_cnt, name_to_start_codes, epoch_ival_ms,
                    min_freq = 0,
                    max_freq=144, last_low_freq=30,
                    low_width=2,
                    high_width=6,
                    low_overlap=0,
                    high_overlap=0,
                    filt_order=3,
                    n_folds=10,
                    n_top_bottom_csp_filters=5,
                    n_selected_filterbands=None,
                    n_selected_features=20,
                    forward_steps=2,
                    backward_steps=1,)

In [ ]:
from braindecode.datautil.trial_segment import create_signal_target, create_signal_target_from_raw_mne

from braindecode.datautil.signalproc import bandpass_cnt

from braindecode.datautil.iterators import get_balanced_batches

In [ ]:
exp.run()